# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#store file path
weather_file = "../WeatherPy/output.csv"
#read file
weather_df = pd.read_csv(weather_file, encoding="ISO-8859-1")
#show the header
weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,lamu,-2.2717,40.9020,82.40,89,20,5.75,KE,1618617474
1,kisangani,0.5167,25.2000,70.12,97,61,2.46,CD,1618617474
2,mataura,-46.1927,168.8643,52.00,76,82,8.01,NZ,1618617474
3,san patricio,28.0170,-97.5169,80.01,83,90,6.91,US,1618617475
4,kodiak,57.7900,-152.4072,46.40,61,75,9.22,US,1618617384


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [9]:
#configure gmaps
gmaps.configure(api_key=g_key)

#use lat and lng and humidity
locations = weather_df[["Lat", "Lng"]]

humidity = weather_df["Humidity"].astype(float)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=20,
                                 point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

NameError: name 'hotel_df' is not defined

In [ ]:
# Add marker layer ontop of heat map


# Display figure
